 1.	Считайте файл и создайте Spark DataFrame

In [1]:
from pyspark.sql import SparkSession

# Создаем SparkSession
spark = SparkSession.builder.appName("CSV to DataFrame").getOrCreate()


csv_file_path = "130/1/1.csv"

df = spark.read.csv(csv_file_path, header=True, inferSchema=True)


df.show()




+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+---+------+
|         feature_1|        feature_2|         feature_3|         feature_4|         feature_5|         feature_6|         feature_7|         feature_8|         feature_9|       feature_10|        feature_11| id|target|
+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+---+------+
|45.531834384241506|46.83396582114066| 91.35670312904635|  45.5062662358562|59.298901864123664|53.208014106424784|40.884734939439575|53.576295742313334| 94.35099858735667|83.27720884117582| 51.97168648767926|  0|     1|
| 39.01399279627978|48.17808812469899|56.848193716775945| 82.52671245077899| 63.03232148309602| 69.82781855216082| 90.99

2.	Выполните преобразование к числовому типу колонки feature_11

In [2]:
from pyspark.sql.functions import col

df = df.withColumn("feature_11", col("feature_11").cast("double"))

df.printSchema()

root
 |-- feature_1: double (nullable = true)
 |-- feature_2: double (nullable = true)
 |-- feature_3: double (nullable = true)
 |-- feature_4: double (nullable = true)
 |-- feature_5: double (nullable = true)
 |-- feature_6: double (nullable = true)
 |-- feature_7: double (nullable = true)
 |-- feature_8: double (nullable = true)
 |-- feature_9: double (nullable = true)
 |-- feature_10: double (nullable = true)
 |-- feature_11: double (nullable = true)
 |-- id: integer (nullable = true)
 |-- target: integer (nullable = true)



3.	Выведите статистические характеристики колонки feature_11

In [3]:
stat = df.describe("feature_11")

stat = stat.withColumn("feature_11", col("feature_11").cast("string"))

stat.show()

+-------+------------------+
|summary|        feature_11|
+-------+------------------+
|  count|               750|
|   mean| 66.22745050997271|
| stddev| 15.84112966591772|
|    min|38.661267623399524|
|    max| 93.61276418162505|
+-------+------------------+



4.	Рассчитайте дополнительно коэффициенты асимметрии и эксцесса.

In [4]:
from pyspark.sql.functions import skewness, kurtosis

# Коэффициент ассиметрии
skew = df.select(skewness("feature_11")).collect()[0][0]

# Коэффициент эксцесса
kurt = df.select(kurtosis("feature_11")).collect()[0][0]




5.  Запишите полученные данные (3,4) в файл JSON

In [6]:
import json

# Создаем новый DataFrame с данными
new_data = [{"skew": skew, "kurt": kurt}]

file_path = "130/1/statistic.json"

# Запись Статистики в JSON файл
stat.toPandas().to_json(file_path, orient="records", lines=True)


with open(file_path, 'a+') as json_file:
    json.dump(new_data, json_file, indent=4)




<class 'list'>
